In [83]:
import pandas as pd
import numpy as np
import sys
from pathlib import Path

# Añadir src al path
src_path = Path.cwd().parent / 'src'
sys.path.append(str(src_path))
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.ensemble import RandomForestRegressor
import warnings
warnings.filterwarnings('ignore')

from utils import *

BASE_DIR = Path.cwd().parent

In [84]:
features = pd.read_csv(f'{BASE_DIR}/data/features.csv')

### Let's see which model we are gonna use to tackle this problem

In [85]:
from sklearn.model_selection import GridSearchCV
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
from lightgbm import LGBMRegressor

# Seleccionar features para el modelo
# feature_columns = [
#     'year_established',
#        'annual_revenue', 'total_payroll', 
#        'num_employees',
#        'num_quotes', 'num_products_requested',
#        'avg_premium', 'min_premium', 'max_premium', 'carrier_diversity', 'premium_range','num_carriers', 'total_quotes',
#        'carrier_concentration', 'product_concentration', 'std_premium','sum_premium',
#        'iqr_premium', 'premium_skewness',  'premium_to_revenue_ratio',
#        'quotes_per_employee', 'quotes_per_million_revenue'
    
#     # One-hot encoded (se generan automáticamente)
#  ] 

excluded_features = ['account_uuid', 'state', 'business_structure','industry', 'subindustry',  'index','account_value','region','premium_range_bins','premium_log','premium_log_bins','premium_range']
feature_columns = list(set(features.columns) - set(excluded_features))
X = features[feature_columns]
y = features['account_value']



# Preparar datos
# Crear bins, por ejemplo 10 percentiles
y_binned = pd.qcut(y, q=10, duplicates='drop')  # divide y en 10 grupos
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42,stratify=y_binned)

# The reason we are applying this log transformation is to reduce the impact of extreme values (outliers) in the target variable as we have seen in the EDA phase. 
# This can help improve the model's performance and stability. However, remember to reverse this transformation when interpreting the model's predictions and calculating
# metrics on the original scale such as RMSE or MAE.
# This is done using the np.expm1 function, which is the inverse of np.log


# y_train = np.log1p(y_train)  # log(1 + y)
# y_val = np.log1p(y_val)  

In [87]:

# Models to compare
models = {
    'XGBoost': XGBRegressor(
        n_estimators=500,
        max_depth=4,
        learning_rate=0.01,
        subsample=0.7,
        colsample_bytree=0.6,
        random_state=42,
        reg_lambda= 0.5, 
        reg_alpha=1.0,
        eval_metric='rmse'
        
    ),
    'Random Forest': RandomForestRegressor(
        n_estimators=500,
        max_depth=7,
        min_samples_split=2,
        min_samples_leaf= 1,
        random_state=42,
        n_jobs=-1
    ),
    'LightGBM': LGBMRegressor(
        n_estimators=500,
        max_depth=7,
        learning_rate=0.01,
        random_state=42,
        reg_lambda= 0.5, 
        reg_alpha=1.0,
        metric='rmse',
        verbosity=-1  
    )
}

results = {}
for name, model in models.items():
    trained_model, rmse, pred = evaluate_model(model, X_train, X_val, y_train, y_val, name)
    results[name] = {'model': trained_model, 'rmse': rmse, 'predictions': pred}

# Seleccionar mejor modelo
best_model_name = min(results.keys(), key=lambda x: results[x]['rmse'])
best_model = results[best_model_name]['model']
print(f"\n Best model: {best_model_name}")


XGBoost:
RMSE: 1290.25
MAE: 448.13
R²: 0.8704

Random Forest:
RMSE: 1331.03
MAE: 412.34
R²: 0.8621

LightGBM:
RMSE: 1393.39
MAE: 450.10
R²: 0.8489

 Best model: XGBoost


In [88]:
# Now, let's check for overfitting by comparing training and validation R² scores
# As per the theory, a significant difference between training and validation R² scores indicates overfitting.
# A model that performs well on training data but poorly on validation data is likely overfitting.

rf_train_score = results['Random Forest']['model'].score(X_train, y_train)
rf_val_score = results['Random Forest']['model'].score(X_val, y_val)
print(f"RF - Train R²: {rf_train_score:.4f}, Val R²: {rf_val_score:.4f}")
print(f"RF - Train R² - Val R²: {rf_train_score - rf_val_score:.4f}")

xgb_train_score = results['XGBoost']['model'].score(X_train, y_train)
xgb_val_score = results['XGBoost']['model'].score(X_val, y_val)
print(f"XGB - Train R²: {xgb_train_score:.4f}, Val R²: {xgb_val_score:.4f}")
print(f"XGB - Train R² - Val R²: {xgb_train_score - xgb_val_score:.4f}")

lgb_train_score = results['LightGBM']['model'].score(X_train, y_train)
lgb_val_score = results['LightGBM']['model'].score(X_val, y_val)
print(f"LGB - Train R²: {lgb_train_score:.4f}, Val R²: {lgb_val_score:.4f}")
print(f"LGB - Train R² - Val R²: {lgb_train_score - lgb_val_score:.4f}")



# We see that the difference between training and validation R² scores is relatively small for all models, 
# indicating that overfitting is not a significant concern in this case. We are gonna take the best permorming model,
#  which is LGBM, and use it for further tuning and final evaluation.

RF - Train R²: 0.9331, Val R²: 0.8621
RF - Train R² - Val R²: 0.0710
XGB - Train R²: 0.9481, Val R²: 0.8704
XGB - Train R² - Val R²: 0.0777
LGB - Train R²: 0.7831, Val R²: 0.8489
LGB - Train R² - Val R²: -0.0658


In [89]:
y.describe()

count      5709.000000
mean       1723.704689
std        3761.789151
min          29.250000
25%         500.000000
50%         697.000000
75%        1438.870000
max      134752.410000
Name: account_value, dtype: float64

In [90]:
# Parameter grid for tuning
param_grid = {
    'n_estimators': [500,600, 700],  # Añadir esto
    'max_depth': [3,4,5],
    'learning_rate': [0.01, 0.02],
    'subsample': [0.7],
    'colsample_bytree': [0.6],
    'reg_lambda': [0.5],
    'reg_alpha': [1.0]
}


search = GridSearchCV(
    estimator=XGBRegressor(random_state=42,eval_metric='rmse',),
    param_grid=param_grid,
    cv=3,
    scoring='r2',
    n_jobs=-1
)




# Fit the search
search.fit(X_train, y_train)

# Best parameters
print("Best parameters:", search.best_params_)
print("Best CV score:", search.best_score_)

Best parameters: {'colsample_bytree': 0.6, 'learning_rate': 0.02, 'max_depth': 3, 'n_estimators': 700, 'reg_alpha': 1.0, 'reg_lambda': 0.5, 'subsample': 0.7}
Best CV score: 0.733830654507369


In [91]:
model = XGBRegressor(**search.best_params_, random_state=42)

In [92]:
trained_model, rmse, pred = evaluate_model(model, X_train, X_val, y_train, y_val, 'XGBoost Tuned')


XGBoost Tuned:
RMSE: 1272.19
MAE: 429.03
R²: 0.8740


In [93]:
xgb_train_score = trained_model.score(X_train, y_train)
xgb_val_score = trained_model.score(X_val, y_val)
print(f"XGB - Train R²: {xgb_train_score:.4f}, Val R²: {xgb_val_score:.4f}")
print(f"XGB - Train R² - Val R²: {xgb_train_score - xgb_val_score:.4f}")

XGB - Train R²: 0.9700, Val R²: 0.8740
XGB - Train R² - Val R²: 0.0960


In [94]:
import pickle
from pathlib import Path
import joblib

# En Jupyter, usar el directorio de trabajo actual
BASE_DIR = Path.cwd().parent

# Save the model

joblib.dump(trained_model,f'{BASE_DIR}/model/xgboost_model.joblib')

print("Model saved successfully!")

Model saved successfully!


In [ ]:
print(f"Number of features: {X_train.shape[1]}")
print(f"Feature names: {X_train.columns.tolist() if hasattr(X_train, 'columns') else 'No names'}")

Number of features: 38
Feature names: ['log_total_payroll', 'year_established', 'total_payroll', 'product_concentration', 'state_premium_sum_encoded', 'subindustry_sum_premium_encoded', 'carrier_concentration', 'business_structure_revenue_encoded', 'premium_per_employee', 'industry_revenue_encoded', 'num_quotes', 'revenue_x_payroll', 'log_annual_revenue', 'premium_to_revenue_ratio', 'total_quotes', 'premium_ratio_max_avg', 'annual_revenue', 'max_x_nquotes', 'state_revenue_encoded', 'num_employees', 'business_structure_premium_sum_encoded', 'industry_sum_premium_encoded', 'num_products_requested', 'iqr_premium', 'premium_per_revenue', 'premium_range', 'avg_x_nproducts', 'premium_per_quote', 'subindustry_revenue_encoded', 'max_premium', 'quotes_per_million_revenue', 'avg_premium', 'sum_premium', 'carrier_diversity', 'min_premium', 'num_carriers', 'quotes_per_employee', 'revenue_per_employee']


In [293]:
sample_features = [15.2, 3.0, 0.5, 0.8, 25000.0, 1200.0, 0.15, 0.7, 12.0, 0.6, 
                   8500.0, 500.0, 0.9, 2.5, 0.4, 3000.0, 85000.0, 0.08, 16.5, 
                   0.12, 25.5, 144.0, 2.8, 102000.0, 0.15, 0.6, 4.0, 0.55, 85.0, 
                   24500.0, 0.75, 12750000.0]

print("Comparing local model vs API predictions...")

# Local prediction
local_prediction = model.predict(np.array(sample_features).reshape(1, -1))[0]
print(f"Local model prediction: {local_prediction}")

Comparing local model vs API predictions...
Local model prediction: 8.119949340820312
